In [ ]:
import os
import textwrap
import warnings

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from cartopy.io import DownloadWarning as CartopyDownloadWarning
from ipywidgets import interact

from canopy_app import config_cases, run, run_config_sens, DEFAULT_POINT_INPUT, REPO

warnings.filterwarnings("ignore", category=CartopyDownloadWarning)

xr.set_options(display_expand_data=False, display_expand_attrs=False)

plt.rcParams.update(
    {
        "figure.autolayout": True,
        "axes.formatter.limits": (-3, 4),
        "axes.formatter.use_mathtext": True,
        "figure.max_open_warning": 0,
    }
)

%matplotlib widget

## 2-D (lat/lon)

### Default case

Southeast US

In [ ]:
inp = xr.open_dataset("../input/gfs.t12z.20220701.sfcf000.canopy.nc")

In [ ]:
inp

In [ ]:
%%time

ds = run()
if "time" in ds.dims:
    ds = ds.isel(time=0)

In [ ]:
proj = ccrs.Mercator()
tran = ccrs.PlateCarree()

fig = plt.figure(constrained_layout=True)
ax = fig.add_subplot(projection=proj)

ax.add_feature(cfeature.STATES, edgecolor="orangered", linewidth=1)
ax.coastlines(color="orangered", linewidth=1.5)
ax.gridlines(draw_labels=True)

ds.waf.plot(
    x="lon", y="lat", ax=ax, transform=tran, cbar_kwargs=dict(orientation="horizontal")
)

In [ ]:
proj = ccrs.Mercator()
tran = ccrs.PlateCarree()

# NOTE: currently flame height and wind speed var names are different in nc vs txt output ds
vns = ["ws", "kz", "rjcf", "emi_isop", "flameh", "waf"]

fig, axs = plt.subplots(
    2,
    3,
    subplot_kw=dict(projection=proj),
    constrained_layout=True,
    figsize=(12, 7),
    sharex=True,
    sharey=True,
)


hc = ds.canheight
dz = ds.z.diff("z")
for i, (vn, ax) in enumerate(zip(vns, axs.flat)):
    ax.add_feature(cfeature.STATES, edgecolor="orangered", linewidth=1)
    ax.coastlines(color="orangered", linewidth=1.5)
    gl = ax.gridlines(draw_labels=True)
    if i % ax.get_gridspec().ncols in range(1, ax.get_gridspec().ncols - 1):
        gl.right_labels = False
        gl.left_labels = False

    da = ds[vn]
    if vn in {"ws", "kz", "rjcf"}:
        # Canopy mean
        da = da.where(da.z <= hc).mean("z")
        agg_label = "canopy mean"
    elif vn in {"emi_isop"}:
        # Integrate
        da = (da * dz).sum("z")
        agg_label = "integral"
        assert ds[vn].attrs["units"] == "kg m-3 s-1"
        da.attrs["units"] = "kg m-2 s-1"
    else:
        assert da.dims == ("y", "x"), "2-D"
        agg_label = None

    cbar_label = vn
    if "units" in da.attrs:
        cbar_label += f" [{da.units}]"
    if agg_label is not None:
        cbar_label += f"\n({agg_label})"
    da.plot(
        x="lon",
        y="lat",
        ax=ax,
        transform=tran,
        cbar_kwargs=dict(orientation="horizontal", label=cbar_label),
    )
    ax.set_title("")

### Canopy thresholds

In [ ]:
%%time

threshes = [
    # lai, canfrac, ch
    (0.1, 0.1, 0.5),  # defaults
    (0.5, 0.5, 0.5),
    (0.1, 0.5, 3),
    (0.1, 0.5, 10),
]

lai_threshes, cf_threshes, ch_threshes = [list(x) for x in zip(*threshes)]

cases = config_cases(
    lai_thresh=lai_threshes,
    cf_thresh=cf_threshes,
    ch_thresh=ch_threshes,
)
print(cases)

ds = run_config_sens(cases)

In [ ]:
def plot_vn_interactive(
    ds, cases, vns=["waf", "ws", "kz", "rjcf", "emi_isop", "flameh"], vn0=None
):
    assert len(vns) >= 0
    if vn0 is None:
        pass
    else:
        vns = vns[:]
        vns.remove(vn0)
        vns.insert(0, vn0)

    hc = ds.canheight
    if "time" in hc.dims:
        hc = hc.isel(time=0)
    dz = ds.z.diff("z")

    proj = ccrs.Mercator()
    tran = ccrs.PlateCarree()

    n = len(cases)
    ncol = 2
    nrow, rem = divmod(n, ncol)
    if rem:
        nrow += 1

    fig = plt.figure(
        constrained_layout=True,
        figsize=(9, 1 + 3.1 * nrow),
    )

    @interact(
        vn=vns,
    )
    def f(vn):
        fig.clf()
        axs = []
        share = None
        for i in range(n):
            share = ax = fig.add_subplot(
                nrow,
                ncol,
                i + 1,
                projection=proj,
                sharex=share,
                sharey=share,
            )
            axs.append(ax)

        da = ds[vn]
        if "time" in da.dims:
            da = da.isel(time=0)
        if vn in {"ws", "kz", "rjcf"}:
            # Canopy mean
            da = da.where(da.z <= hc).mean("z")
            agg_label = "canopy mean"
        elif vn in {"emi_isop"}:
            # Integrate
            da = (da * dz).sum("z")
            agg_label = "integral"
            assert ds[vn].attrs["units"] == "kg m-3 s-1"
            da.attrs["units"] = "kg m-2 s-1"
        else:
            assert da.isel(case=0).dims == ("y", "x"), "2-D"
            agg_label = None

        dq = 0.05
        vmin, vmax = da.quantile([dq, 1 - dq]).values

        cbar_label = vn
        if "units" in da.attrs:
            cbar_label += f" [{da.units}]"
        if agg_label is not None:
            cbar_label += f"\n({agg_label})"

        for i, (case, ax) in enumerate(zip(cases, axs)):
            ax.add_feature(cfeature.STATES, edgecolor="orangered", linewidth=1)
            ax.coastlines(color="orangered", linewidth=1.5)
            gl = ax.gridlines(draw_labels=True)
            if i % 2 == 1:
                gl.right_labels = False
                gl.left_labels = False

            im = da.isel(case=i).plot.pcolormesh(
                x="lon",
                y="lat",
                transform=tran,
                vmin=vmin,
                vmax=vmax,
                add_colorbar=False,
                ax=ax,
            )

            ax.set_title(
                ", ".join(f"{k}={v}" for k, v in case["userdefs"].items()),
                color="forestgreen",
                size=11,
            )

        fig.colorbar(im, ax=axs, orientation="horizontal", shrink=0.7, label=cbar_label)

In [ ]:
plot_vn_interactive(ds, cases)

### Flame height options

In [ ]:
%%time

opts = [
    # flameh_opt, flameh_set, frp_fac
    (0, 1.0, 1.0),
    # ^ (Straight FRP to flameh calculation, only for active FRP/fire points, no FRP tuning)
    (1, 2.0, 1.0),
    # ^ (User-set flame height = 2.0 m for all canopy points)
    (2, 2.0, 1.0),
    # ^ (FRP to flameh calculation for active FRP/fire points, user-set flame height = 2.0 m elsewhere, no FRP tuning)
    (3, 0.5, 1.0),
    # ^ (Flameh is set to 0.5*Hc (m) for all canopy points)
    (4, 0.5, 1.0),
    # ^ (FRP to flameh calculation for active FRP/fire points, flameh is set to 0.5*Hc (m) elsewhere, no FRP tuning)
    (5, 0.5, 10.0),
    # ^ (FRP to flameh calculation for active FRP/fire points with crowning dependence (flameh=Hc),
    #   flameh is set to 0.5*Hc (m) elsewhere, FRP increased by a factor of 10)
]

flameh_opts, flameh_sets, frp_facs = [list(x) for x in zip(*opts)]

cases = config_cases(
    flameh_opt=flameh_opts,
    flameh_set=flameh_sets,
    frp_fac=frp_facs,
)
print(cases)

ds = run_config_sens(cases)

In [ ]:
plot_vn_interactive(ds, cases)

### PAI options

In [ ]:
%%time

opts = [
    # pai_opt, pai_set
    (0, 1),
    # ^ (PAI from Katul et al., 2004 vegetation types/lookup table)
    (2, 1),
    # ^ (PAI estimated from model LAI)
    (3, 1),
    # ^ (User-set PAI = 1)
    (3, 2),
    # ^ (User-set PAI = 2)
    (3, 4),
    # ^ (User-set PAI = 4)
]

pai_opts, pai_sets = [list(x) for x in zip(*opts)]

cases = config_cases(
    pai_opt=pai_opts,
    pai_set=pai_sets,
)
print(cases)

ds = run_config_sens(cases)

In [ ]:
plot_vn_interactive(ds, cases)

### Canopy environment coefficient (biogenics)

In [ ]:
%%time

opts = [0.1, 0.2, 0.5, 0.8, 0.9]

cases = config_cases(
    bio_cce=opts,
)
print(cases)

ds = run_config_sens(cases)

In [ ]:
plot_vn_interactive(ds, cases, vn0="emi_isop")

## 1-D (lat/lon point)

### Default case

In [ ]:
%%time

ds = run(
    config={
        "filenames": {"file_vars": "../input/point_file_20220701.sfcf000.txt"},
        "userdefs": {"infmt_opt": 1, "ntime": 1, "nlat": 1, "nlon": 1},
    },
)

In [ ]:
vns = ["ws", "kz", "rjcf", "emi_isop"]

fig, axs = plt.subplots(1, len(vns), figsize=(11, 4), sharey=True)

for vn, ax in zip(vns, axs.flat):
    ax.axhline(ds.canheight, c="0.5", ls="--")
    ds[vn].plot(y="z", ax=ax)
    ax.set_title(
        "\n".join(textwrap.wrap(ax.get_title(), width=30, break_long_words=False)),
        size=12,
    )

ax.set_ylim(ymin=0);

In [ ]:
(
    ds[[vn for vn in ds.data_vars if vn.startswith("emi_")]]
    .sel(z=6, method="nearest")
    .to_pandas()
    .to_frame()
    .plot.bar(figsize=(6, 3), legend=False)
)
plt.ylabel(ds["emi_isop"].units)
plt.yscale("log")

### $z_0 / h_c$

The namelist parameter `z0ghc` represents the ratio of ground roughness length to canopy top height, i.e.
$z_0 / h_c$. In general, they increase together, but this is still a tunable parameter.

In [ ]:
%%time

cases = config_cases(
    file_vars="../input/point_file_20220701.sfcf000.txt",
    infmt_opt=1,
    ntime=1,
    nlat=1,
    nlon=1,
    z0ghc=np.power(10, np.linspace(-4, np.log10(0.25), 100)).tolist(),
    product=False,
)
ds = run_config_sens(cases)
ds

In [ ]:
hc = ds.canheight.isel(case=0)
assert (ds.canheight == hc).all()
ds_c = ds.isel(z=ds.z <= hc).mean("z")  # canopy mean

vns = ["ws", "kz", "waf"]

fig, axs = plt.subplots(1, len(vns), figsize=(9, 3.5), sharey=False)

fig.suptitle("Canopy mean")

for vn, ax in zip(vns, axs.flat):
    ax.plot(ds_c["z0ghc"], ds_c[vn])
    ax.set(ylabel=f"{vn} [{ds[vn].units}]", xlabel="$z_0 / h_c$")
    ax.set_xscale("log")

### Canopy environment coefficient (biogenics)

In [ ]:
cce_base = 0.21

fig = plt.figure(figsize=(8, 4))


@interact(
    cce=(0.0, 1.0, 0.01),
)
def f(cce=0.57):  # 0.57 in MEGAN2.1 paper
    fig.clf()
    ax = fig.add_subplot()

    cases = config_cases(
        file_vars="../input/point_file_20220701.sfcf000.txt",
        infmt_opt=1,
        ntime=1,
        nlat=1,
        nlon=1,
        bio_cce=[cce_base, cce],
        product=False,
    )
    ds = run_config_sens(cases)

    hc = ds.canheight.isel(case=0)
    assert (ds.canheight == hc).all()
    dz = ds.z.diff("z")
    (
        # ds[[vn for vn in ds.data_vars if vn.startswith("emi_")]].sel(z=10, method="nearest")
        # (ds[[vn for vn in ds.data_vars if vn.startswith("emi_")]].isel(z=ds.z <= hc) * dz).sum("z")
        (ds[[vn for vn in ds.data_vars if vn.startswith("emi_")]] * dz)
        .sum("z")
        .rename(case="CCE")
        .to_pandas()
        .drop(columns=["time", "z", "lat", "lon"], errors="ignore")
        .T.rename(columns={0: f"default ({cce_base})", 1: cce})
        .plot.bar(figsize=(6, 3), legend=True, ax=ax)
    )
    ax.set_yscale("log")
    ax.set_ylabel(ds["emi_isop"].units)

### Input variables

We can create different point input files based on the default case and run the model for each,
allowing us to examine the sensitivity to the input variables.

Note that files matching `test_*.txt` are gitignored.

In [ ]:
DEFAULT_POINT_INPUT.T.rename(columns={0: "value"})

In [ ]:
%%time

# Create input files
clu = np.linspace(0.3, 1.0, 50)
df = DEFAULT_POINT_INPUT.copy()
paths = []
for i, clu_i in enumerate(clu):
    df["clu"] = clu_i
    p = REPO / "input" / f"test_point_{i:02}.txt"
    paths.append(p)
    df.to_csv(p, index=False)

# Run cases
cases = config_cases(
    file_vars=[p.as_posix() for p in paths],
    infmt_opt=1,
    ntime=1,
    nlat=1,
    nlon=1,
    product=False,
)
ds = run_config_sens(cases)

# Clean up input files
for p in paths:
    os.remove(p)

In [ ]:
hc = ds.canheight.isel(case=0)
assert (ds.canheight == hc).all()
ds_c = ds.isel(z=ds.z <= hc).mean("z")  # canopy mean

vns = ["rjcf", "emi_isop", "emi_apin"]

fig, axs = plt.subplots(1, len(vns), figsize=(9, 3.5), sharey=False)

fig.suptitle("Canopy mean")

for vn, ax in zip(vns, axs.flat):
    ax.plot(clu, ds_c[vn])
    ax.set(ylabel=f"{vn} [{ds[vn].units}]", xlabel="CLU")

### Emissions calculation options

In [ ]:
cases = config_cases(
    file_vars="../input/point_file_20220701.sfcf000.txt",
    infmt_opt=1,
    ntime=1,
    nlat=1,
    nlon=1,
    biovert_opt=[0, 1, 2, 3],
)
ds = run_config_sens(cases)
ds

Currently, method 1 should give the same result as canopy-integrating the laywerwise method 0 results afterwards.

The method 2 result is smaller since it places more weight on the lower--middle regions of the canopy, where light levels are lower. Method 3 is somewhere in between these (consider the shape of the LAD profile vs a Gaussian in height).

In [ ]:
dz = ds.z.diff("z")

e0 = float((ds.emi_isop.isel(case=0) * dz).sum("z"))
e1 = float(ds.emi_isop.isel(case=1, z=-1))
e2 = float(ds.emi_isop.isel(case=2, z=-1))
e3 = float(ds.emi_isop.isel(case=3, z=-1))
print(e0, e1, e2, e3, sep="\n")

assert np.isclose(e0, e1)

### Multiple time steps

In [ ]:
%%time

ds = run(
    config={
        "filenames": {
            "file_vars": [
                "../input/point_file_20220630.sfcf023.txt",
                "../input/point_file_20220701.sfcf000.txt",
                "../input/point_file_20220701.sfcf001.txt",
            ],
        },
        "userdefs": {
            "infmt_opt": 1,
            "ntime": 3,
            "nlat": 1,
            "nlon": 1,
        },
    },
)
ds

In [ ]:
ds.emi_apin.mean("z", keep_attrs=True).plot(size=4, marker="o")

### Leaf age parameterization

In [ ]:
config = {
    "filenames": {
        "file_vars": [
            "../input/point_file_20220630.sfcf023.txt",
            "../input/point_file_20220701.sfcf000.txt",
            "../input/point_file_20220701.sfcf001.txt",
        ],
    },
    "userdefs": {
        "infmt_opt": 1,
        "ntime": 3,
        "nlat": 1,
        "nlon": 1,
        "leafage_opt": 0,  # on
    },
}
ds0 = run(config=config)

config["userdefs"]["leafage_opt"] = 1  # off
ds1 = run(config=config)

ds = xr.concat([ds0, ds1], dim="leafage_opt")
ds

In [ ]:
fig, [ax1, ax2] = plt.subplots(2, 1, figsize=(5, 6), sharex=True)

ds.emi_apin.mean("z", keep_attrs=True).plot(hue="leafage_opt", marker="o", ax=ax1)
ds.emi_isop.mean("z", keep_attrs=True).plot(hue="leafage_opt", marker="o", ax=ax2)